# CamelAI + TiDB Demo

### Install dependencies

In [ ]:
!rm -rf .venv/lib/python3.11/site-packages/camel*
!uv cache clean camel-ai
!uv add ../camel
!uv sync

### Load the environment variables

1. Go to [TiDB Cloud](https://tidbcloud.com/console/clusters)
2. Create a serverless cluster in a few seconds
3. Setup the environment variables, add the tidb database url and OpenAI API Key to the `.env` file:

In [ ]:
!cp .env.example .env
!vi .env

In [14]:
import os
from dotenv import load_dotenv

load_dotenv()

True

### Initialize the memory with TiDB

In [15]:
from camel.memories import (
    ChatHistoryBlock,
    LongtermAgentMemory,
    ScoreBasedContextCreator,
    VectorDBBlock,
)
from camel.types import ModelType, OpenAIBackendRole
from camel.utils import OpenAITokenCounter
from camel.embeddings import OpenAIEmbedding
from camel.storages import TiDBStorage

embedding =  OpenAIEmbedding()
memory = LongtermAgentMemory(
    context_creator=ScoreBasedContextCreator(
        token_counter=OpenAITokenCounter(ModelType.GPT_4O_MINI),
        token_limit=1024,
    ),
    chat_history_block=ChatHistoryBlock(),
    vector_db_block=VectorDBBlock(
        storage=TiDBStorage(
            url_and_api_key=os.environ["DATABASE_URL"],
            vector_dim=embedding.get_output_dim()
        )
    ),
)

### Add records to the memory

In [9]:
from camel.messages import BaseMessage
from camel.memories import MemoryRecord

records = [
    MemoryRecord(
        message=BaseMessage.make_user_message(
            role_name="User",
            meta_dict=None,
            content="What is CAMEL AI?",
        ),
        role_at_backend=OpenAIBackendRole.USER,
    ),
    MemoryRecord(
        message=BaseMessage.make_assistant_message(
            role_name="Agent",
            meta_dict=None,
            content="CAMEL-AI.org is the 1st LLM multi-agent framework and "
                    "an open-source community dedicated to finding the scaling law "
                    "of agents.",
        ),
        role_at_backend=OpenAIBackendRole.ASSISTANT,
    ),
]
memory.write_records(records)

### Retrieve the context from the memory

In [10]:
context, token_count = memory.get_context()

print(f"Retrieved context (token count: {token_count}):")
for message in context:
    print(f"{message}")

Retrieved context (token count: 49):
{'role': 'user', 'content': 'What is CAMEL AI?'}
{'role': 'assistant', 'content': 'CAMEL-AI.org is the 1st LLM multi-agent framework and an open-source community dedicated to finding the scaling law of agents.'}


### Chat with the agent

In [16]:
from camel.agents import ChatAgent

# Define system message for the agent
sys_msg = BaseMessage.make_assistant_message(
    role_name='Agent',
    content='You are a curious agent wondering about the universe.',
)

# Initialize agent
agent = ChatAgent(
    system_message=sys_msg,
    memory=memory             # 👈 Add the memory to the agent.
)

# Define a user message
usr_msg = BaseMessage.make_user_message(
    role_name='User',
    content="Tell me which is the 1st LLM multi-agent framework based on what we have discussed",
)

# Sending the message to the agent
response = agent.step(usr_msg)

# Check the response (just for illustrative purpose)
print(response.msgs[0].content)

As of my last knowledge update in October 2023, one of the notable early frameworks for multi-agent systems utilizing large language models (LLMs) is **LangChain**. LangChain allows developers to build applications that can leverage LLMs in a multi-agent context, enabling agents to interact, collaborate, and perform tasks collectively.

If there are any specific frameworks or developments that have emerged after that date, I would not be aware of them. For the most current information, I recommend checking recent publications or announcements in the field of AI and multi-agent systems.
